In [1]:
import numpy as np; import pandas as pd
from GANcc_shim import GAN_SMILES as GANcc #~ conditional GAN (Y=0,oneC)
import math
import tensorflow as tf
from rdkit import Chem
from rdkit.Chem import Descriptors
tf.reset_default_graph()

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"


In [6]:
raw = pd.read_csv('data/ask1_re_1208.csv', delimiter=',', header=0) #ask1_635_descriptor.csv
raw = raw.values # type(df) : numpy.ndarray

xc=raw[:,5].tolist() #1
logP0=raw[:,6] #5

In [8]:
xc

['[O-][n+]1ccc2c(-c3ccc(F)cc3Cl)ccnc2c1-c1c(Cl)cccc1Cl',
 'C=CCOCCOc1cc(-c2cn[nH]c2)ccc1N=C(O)C1COc2ccc(OC)cc2C1',
 'C=CCOCCOc1cc(-c2cn[nH]c2)ccc1NC(=O)C1COc2ccc(OC)cc2C1',
 'c1cc(-n2cnc3c(NC4CC4)nc(Nc4ccc(N5CCOCC5)cc4)nc32)cs1',
 'c1cc(-n2cnc3c(Nc4ccc(N5CCOCC5)cc4)[nH]c(=NCc4ccc5c(c4)OCO5)nc32)cs1',
 'c1cc(-n2cnc3c(Nc4ccc(N5CCOCC5)cc4)nc(N4CCN(Cc5ccc6c(c5)OCO6)CC4)nc32)cs1',
 'c1cc(-n2cnc3c(Nc4ccc(N5CCOCC5)cc4)nc(NCc4ccc5c(c4)OCO5)nc32)cs1',
 'c1cc(-n2cnc3c(Nc4ccc5c(c4)OCO5)nc(N4CCN(Cc5ccc6c(c5)OCO6)CC4)nc32)cs1',
 'c1cc(-n2cnc3c(Nc4ccc5c(c4)OCO5)nc(Nc4ccc(N5CCOCC5)cc4)nc32)cs1',
 'c1cc(=Nc2ccc3c(c2)OCO3)[nH]c(N2CCN(Cc3ccc4c(c3)OCO4)CC2)n1',
 'c1cc(=Nc2ccc3c(c2)OCO3)[nH]c(Nc2ccc(N3CCOCC3)cc2)n1',
 'c1cc(Nc2ccc3c(c2)OCO3)nc(N2CCN(Cc3ccc4c(c3)OCO4)CC2)n1',
 'c1cc(Nc2ccc3c(c2)OCO3)nc(Nc2ccc(N3CCOCC3)cc2)n1',
 'c1ccc(-n2cnc3c(Nc4ccc(N5CCOCC5)cc4)[nH]c(=Nc4cccnc4)nc32)cc1',
 'c1ccc(-n2cnc3c(Nc4ccc(N5CCOCC5)cc4)nc(Nc4cccnc4)nc32)cc1',
 'c1ccc(-n2cnc3c(NCc4cccnc4)nc(Nc4ccc(N5CCOCC5)cc4)nc32)

In [19]:
import random; random.seed(0); random.shuffle(xc)
random.seed(0); random.shuffle(logP0)


y=np.zeros(len(xc), np.int64) 
y[np.where(pd.qcut(logP0,5,labels = [0,1,2,3,4], retbins=False)==1)]=1 
y[np.where(pd.qcut(logP0,5,labels = [0,1,2,3,4], retbins=False)==2)]=2 
y[np.where(pd.qcut(logP0,5,labels = [0,1,2,3,4], retbins=False)==3)]=3 
y[np.where(pd.qcut(logP0,5,labels = [0,1,2,3,4], retbins=False)==4)]=4 


#3등분
cutoff=pd.qcut(logP0,5,labels = [0,1,2,3,4], retbins=False)


v0=np.where(cutoff==0)  
v1=np.where(cutoff==1)
v2=np.where(cutoff==2)
v3=np.where(cutoff==3)  
v4=np.where(cutoff==4)



data='\n'.join(xc)

In [20]:
pd.DataFrame(y).iloc[:,0].value_counts()

4    89
1    89
0    89
3    88
2    88
Name: 0, dtype: int64

In [21]:
pd.DataFrame(list(cutoff)).iloc[:,0].value_counts()

4    89
1    89
0    89
3    88
2    88
Name: 0, dtype: int64

In [22]:
 np.mean(logP0[v0]),np.mean(logP0[v1]),np.mean(logP0[v2]) ,np.mean(logP0[v3]),np.mean(logP0[v4]) #,np.mean(logP0[v5]),np.mean(logP0[v6]),np.mean(logP0[v7]),np.mean(logP0[v8])

(2.689594831460674,
 3.7566080898876404,
 4.410206136363635,
 5.075958181818184,
 6.047653483146067)

In [24]:
d10=np.mean(logP0[v1])-np.mean(logP0[v0]); d21=np.mean(logP0[v2])-np.mean(logP0[v1]); d32=np.mean(logP0[v3])-np.mean(logP0[v2]);  d43=np.mean(logP0[v4])-np.mean(logP0[v3])
#d54=np.mean(logP0[v5])-np.mean(logP0[v4]);  d65=np.mean(logP0[v6])-np.mean(logP0[v5]); d76=np.mean(logP0[v6])-np.mean(logP0[v6]);  d87=np.mean(logP0[v8])-np.mean(logP0[v7])

oneC=math.ceil((1/min(d10,d21,d32,d43))*15)

In [25]:
oneC

23

In [28]:
#GANcc 
nitr=700
n_samplings=800; learning_rate =5e-5;batch_size =16; epochs =1000  
xchange=[['Br','R'],['Cl','L']]   


gen_samplesFcc=[[] for i in range(5)] # list of dim 3
for k in range(5):
    yCLASS=k 
    gen_samplesFc=[]
    for ii in range(nitr):
        print('itr=',ii+1)
        gen_samplesF0cc=GANcc(data,oneC,y,yCLASS, n_samplings, learning_rate,batch_size, epochs,xchange)         
        if len(gen_samplesF0cc)>0:
            for i in range(len(gen_samplesF0cc)):
                gen_samplesFc.append(gen_samplesF0cc[i])                 
    gen_samplesFc=set(gen_samplesFc) & set(gen_samplesFc)
    gen_samplesFc=list(gen_samplesFc)            
    gen_samplesFc.sort()               
    gen_samplesFcc[k]=gen_samplesFc 

       
logPcc=[[] for i in range(5)]; gen_samplesFccNEW=[[] for i in range(3)]  
for k in range(5):
    for i in range(len(gen_samplesFcc[k])):
        try:
            m=Chem.MolFromSmiles(gen_samplesFcc[k][i])
            b=Descriptors.MolLogP(m)
        except ValueError: continue
        logPcc[k].append(b); 
        if np.asarray(gen_samplesFcc[k]).shape[0]>0: gen_samplesFccNEW[k].append(gen_samplesFcc[k][i])
        else: gen_samplesFccNEW[k]=gen_samplesFcc[k]

    
print('\n mean(whole, each class)', round(np.mean(logP0),3),round(np.mean(logP0[v0]),3),
      round(np.mean(logP0[v1]),3),round(np.mean(logP0[v2]),3))    
print('\n n of [GANcc[0]-GANcc[1]-GANcc[2] ]: \n',len(gen_samplesFcc[0]),len(gen_samplesFcc[1]),len(gen_samplesFcc[2]),len(gen_samplesFcc[3]),len(gen_samplesFcc[4])) #,len(gen_samplesFcc[3]),len(gen_samplesFcc[4])
print(' n of [new(GANcc[0]-GANcc[1]-GANcc[2]) ]: \n',len(gen_samplesFccNEW[0]),len(gen_samplesFccNEW[1]),len(gen_samplesFccNEW[2]),len(gen_samplesFccNEW[3]),len(gen_samplesFccNEW[4])) #,len(gen_samplesFccNEW[3]),len(gen_samplesFccNEW[4])
print(' mean logP of [GANcc[0]-GANcc[1]-GANcc[2] ]: \n',
      round(np.mean(logPcc[0]),3),round(np.mean(logPcc[1]),3),round(np.mean(logPcc[2]),3),round(np.mean(logPcc[3]),3),round(np.mean(logPcc[4]),3)) #,round(np.mean(logPcc[3]),3),round(np.mean(logPcc[4]),3)

#[ 70 22 4]
#[ 70 21 4]
#[ 0.977 3.749 5.536]



itr= 1
No valid SMILES structure is generated.
itr= 2
No valid SMILES structure is generated.
itr= 3
No valid SMILES structure is generated.
itr= 4
No valid SMILES structure is generated.
itr= 5
No valid SMILES structure is generated.
itr= 6
No valid SMILES structure is generated.
itr= 7
No valid SMILES structure is generated.
itr= 8
No valid SMILES structure is generated.
itr= 9
No valid SMILES structure is generated.
itr= 10
No valid SMILES structure is generated.
itr= 11
No valid SMILES structure is generated.
itr= 12
No valid SMILES structure is generated.
itr= 13
No valid SMILES structure is generated.
itr= 14
No valid SMILES structure is generated.
itr= 15
No valid SMILES structure is generated.
itr= 16
No valid SMILES structure is generated.
itr= 17
No valid SMILES structure is generated.
itr= 18
No valid SMILES structure is generated.
itr= 19
No valid SMILES structure is generated.
itr= 20
No valid SMILES structure is generated.
itr= 21
No valid SMILES structure is generated.
i

No valid SMILES structure is generated.
itr= 172
No valid SMILES structure is generated.
itr= 173
No valid SMILES structure is generated.
itr= 174
No valid SMILES structure is generated.
itr= 175
No valid SMILES structure is generated.
itr= 176
No valid SMILES structure is generated.
itr= 177
No valid SMILES structure is generated.
itr= 178
No valid SMILES structure is generated.
itr= 179
No valid SMILES structure is generated.
itr= 180
No valid SMILES structure is generated.
itr= 181
No valid SMILES structure is generated.
itr= 182
No valid SMILES structure is generated.
itr= 183
No valid SMILES structure is generated.
itr= 184
No valid SMILES structure is generated.
itr= 185
No valid SMILES structure is generated.
itr= 186
No valid SMILES structure is generated.
itr= 187
No valid SMILES structure is generated.
itr= 188
No valid SMILES structure is generated.
itr= 189
No valid SMILES structure is generated.
itr= 190
No valid SMILES structure is generated.
itr= 191
No valid SMILES stru

No valid SMILES structure is generated.
itr= 340
No valid SMILES structure is generated.
itr= 341
No valid SMILES structure is generated.
itr= 342
No valid SMILES structure is generated.
itr= 343
No valid SMILES structure is generated.
itr= 344
No valid SMILES structure is generated.
itr= 345
No valid SMILES structure is generated.
itr= 346
No valid SMILES structure is generated.
itr= 347
No valid SMILES structure is generated.
itr= 348
No valid SMILES structure is generated.
itr= 349
No valid SMILES structure is generated.
itr= 350
No valid SMILES structure is generated.
itr= 351
No valid SMILES structure is generated.
itr= 352
No valid SMILES structure is generated.
itr= 353
No valid SMILES structure is generated.
itr= 354
No valid SMILES structure is generated.
itr= 355
No valid SMILES structure is generated.
itr= 356
No valid SMILES structure is generated.
itr= 357
No valid SMILES structure is generated.
itr= 358
No valid SMILES structure is generated.
itr= 359
No valid SMILES stru

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(gen_samplesFccNEW[0]).to_csv('~/projects/pd1/data/ask1_cgan_logp1_1208.csv',index=None,header=None)
pd.DataFrame(gen_samplesFccNEW[1]).to_csv('~/projects/pd1/data/ask1_cgan_logp2_1208.csv',index=None,header=None)
pd.DataFrame(gen_samplesFccNEW[2]).to_csv('~/projects/pd1/data/ask1_cgan_logp3_1208.csv',index=None,header=None)
pd.DataFrame(gen_samplesFccNEW[3]).to_csv('~/projects/pd1/data/ask1_cgan_logp4_1208.csv',index=None,header=None)
pd.DataFrame(gen_samplesFccNEW[4]).to_csv('~/projects/pd1/data/ask1_cgan_logp5_1208.csv',index=None,header=None)